# Get Candlestick Patterns

## Import relevant libraries

In [1]:
import os
import pandas as pd
import talib
import mplfinance as mpf
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Get the data

In [2]:
input_data_path = '../data'
input_data_filename = 'binance_BTCUSDT_1m_from_2020_01_01_to_2021_12_31'
input_data_extension = ".csv"
full_path_input_data = os.path.join(input_data_path, input_data_filename + input_data_extension)

In [3]:
df = pd.read_csv(full_path_input_data)

In [4]:
df.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volumne', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore',
       'formatted_open_time', 'formatted_close_time', 'upper_shadow',
       'lower_shadow', 'real_body'],
      dtype='object')

## Identify Candlestick Patterns

In [5]:
candle_names = talib.get_function_groups()['Pattern Recognition']

- Inverted Hammer --> Single [Bullish]
- Hammer --> Single [Bullish]
- Bullish Engulfing (not available at ta-lib) --> Double [Bullish]
- Piercing Pattern --> Double [Bullish]
- Morning Star --> Triple [Bullish]


- Shooting Star --> Single [Bearish]
- Hanging Man --> Single [Bearish]
- Bearish Engulfing (not available at ta-lib) --> Double [Bearish]
- Dark Cloud Cover --> Double [Bearish]
- Evening Star --> Triple [Bearish]

- Engulfing [positive: Bullish / negative: Bearish]

In [6]:
candle_names = [
    'CDLINVERTEDHAMMER',
    'CDLHAMMER',
    'CDLPIERCING',
    'CDLMORNINGSTAR',
    'CDLSHOOTINGSTAR',
    'CDLHANGINGMAN',
    'CDLDARKCLOUDCOVER',
    'CDLEVENINGSTAR',
    'CDLENGULFING'
]

In [7]:
# extract OHLC 
op = df['open']
hi = df['high']
lo = df['low']
cl = df['close']

# create columns for each pattern
for candle in candle_names:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    df[candle] = getattr(talib, candle)(op, hi, lo, cl)

In [8]:
df.sample(10)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,real_body,CDLINVERTEDHAMMER,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING
290286,1595308080000,9182.64,9182.66,9181.14,9181.63,6.626332,1595308139999,6.084092e+04,277,1.644472,...,1.01,0,0,0,0,0,0,0,0,0
626811,1615530720000,57201.42,57221.56,57187.24,57193.19,27.816407,1615530779999,1.591297e+06,899,15.500330,...,8.23,0,0,0,0,0,0,0,0,0
469487,1606060140000,18309.99,18316.00,18294.73,18302.78,32.174409,1606060199999,5.889064e+05,846,9.038525,...,7.21,0,0,0,0,0,0,0,0,0
839764,1628333940000,43633.33,43665.50,43622.88,43658.61,20.308790,1628333999999,8.863002e+05,831,10.962690,...,25.28,0,0,0,0,0,0,0,0,0
419165,1603040820000,11447.72,11451.51,11447.72,11449.00,10.889348,1603040879999,1.246803e+05,260,5.373037,...,1.28,0,0,0,0,-100,0,0,0,0
738107,1622234520000,35479.60,35491.70,35384.96,35427.39,392.998622,1622234579999,1.391497e+07,3019,289.629364,...,52.21,0,0,0,0,0,0,0,0,-100
181778,1588785000000,9258.42,9258.71,9250.00,9252.98,36.673181,1588785059999,3.393443e+05,328,10.632402,...,5.44,0,0,0,0,0,0,0,0,0
222658,1591237800000,9672.51,9684.89,9672.01,9677.33,65.102468,1591237859999,6.302301e+05,866,31.746404,...,4.82,0,0,0,0,0,0,0,0,0
334439,1597957260000,11853.19,11854.97,11852.14,11853.35,37.677651,1597957319999,4.465852e+05,647,14.888880,...,0.16,0,0,0,0,0,0,0,0,0
300581,1595925780000,10808.14,10808.56,10801.18,10804.60,44.935957,1595925839999,4.855477e+05,500,14.908820,...,3.54,0,0,0,0,0,0,0,0,0


## Split 'CDLENGULFING' into Bullish and Bearish one

In [9]:
list_open_time_bullish_engulfing = df[df['CDLENGULFING']==100]['open_time'].to_list()
list_open_time_bearish_engulfing = df[df['CDLENGULFING']==-100]['open_time'].to_list()

In [10]:
df['CDLENGULFINGBULLISH'] = np.where(df['open_time'].isin(list_open_time_bullish_engulfing), 100, 0)
df['CDLENGULFINGBEARISH'] = np.where(df['open_time'].isin(list_open_time_bearish_engulfing), -100, 0)

### Check

In [11]:
df[df['CDLENGULFING']==100].sample(5)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING,CDLENGULFINGBULLISH,CDLENGULFINGBEARISH
338857,1598222340000,11645.82,11647.51,11645.81,11647.32,7.505953,1598222399999,8.741944e+04,178,4.600005,...,0,0,0,0,0,0,0,100,100,0
287005,1595111220000,9168.03,9168.94,9167.62,9168.85,5.684524,1595111279999,5.211676e+04,135,1.931211,...,0,0,0,0,0,0,0,100,100,0
118855,1585000620000,6410.91,6432.90,6410.00,6425.99,200.451747,1585000679999,1.287521e+06,959,106.474813,...,0,0,0,0,0,0,0,100,100,0
963664,1635791340000,60581.81,60625.09,60572.37,60622.54,12.660830,1635791399999,7.671787e+05,722,8.021120,...,0,0,0,0,0,0,0,100,100,0
428282,1603587840000,13113.35,13119.92,13113.35,13117.18,16.539013,1603587899999,2.169406e+05,352,3.668986,...,0,0,0,0,0,0,0,100,100,0


In [12]:
df[df['CDLENGULFING']==-100].sample(5)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING,CDLENGULFINGBULLISH,CDLENGULFINGBEARISH
655238,1617236340000,58994.20,59022.43,58852.94,58883.57,79.411660,1617236399999,4.680509e+06,1701,14.870747,...,0,0,0,0,0,0,0,-100,0,-100
118866,1585001280000,6419.12,6427.77,6403.42,6406.64,84.251408,1585001339999,5.408400e+05,637,20.855382,...,0,0,0,0,0,0,0,-100,0,-100
1003629,1638189240000,56954.05,56954.05,56917.50,56924.41,25.866160,1638189299999,1.472782e+06,685,4.487250,...,0,0,0,0,0,0,0,-100,0,-100
424595,1603366620000,12955.40,12958.19,12917.00,12938.00,228.476840,1603366679999,2.956057e+06,1824,77.729444,...,0,0,0,0,0,0,0,-100,0,-100
895979,1631723040000,48334.69,48334.69,48281.68,48281.74,38.911460,1631723099999,1.879706e+06,1244,8.858200,...,0,0,0,0,0,0,0,-100,0,-100


## Export the Data

### Export parameters

In [13]:
export_path = "../data"
export_filename = input_data_filename + \
                  "_candlesticks_signals_raw"
export_extension = ".csv"
full_export_path = os.path.join(export_path, export_filename + export_extension)

### Export

In [14]:
df.to_csv(full_export_path, index=False)